In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="darkgrid")

# CSV einlesen (Achtung keine / im Namen wegen späterem Namen)
# geht davon aus, dass keine Fehler in der Datei sind und alle Felder richtig gefüllt
df = pd.read_csv('input/20190215LPS_CFD.csv', sep=';', usecols=[9,11,18], parse_dates=[1,2], encoding='iso8859_15')

df.columns = ['art', 'input', 'out']

feiertage = ['2018-10-03', # Tag der Einheit
             '2018-11-01', # Allerheiligen
             '2018-12-24', # Heiligabend
             '2018-12-25', # Weihnachten
             '2018-12-26', # Weihnachten
            ]

# astype('datetime64[D]') notwendig
dauer=np.busday_count(df.input.values.astype('datetime64[D]'), 
                  df.out.values.astype('datetime64[D]'), 
                    holidays=feiertage)+1

df['dauer'] = dauer

von=(min(df.input))
von=von.strftime('%d.%m.%Y')

bis=(max(df.out))
bis=bis.strftime('%d.%m.%Y')

def calc_8090_percentile(ldf, art, von, bis):
    #interpolation noch überprüfen
    perc80 = np.percentile(ldf.dauer,80,interpolation='lower')
    perc90 = np.percentile(ldf.dauer,90,interpolation='lower')
    return (perc80, perc90)
#end 8090_percentile
    

def plot_leadtime(ldf, art, von, bis):
    #interpolation noch überprüfen
    perc80 = np.percentile(ldf.dauer,80,interpolation='lower')
    perc90 = np.percentile(ldf.dauer,90,interpolation='lower')
    
    # gca stands for 'get current axis'
    ax = plt.gca()
    #hist zählt direkt die Elemente
    ldf.plot(kind='hist', y='dauer',bins=max(dauer),rwidth=1, color='black', ax=ax, label='')
    #df[gewaehlt].plot(kind='hist', y='dauer',bins=max(dauer),rwidth=1, color='black', ax=ax, title='selected work item type')

    text90 = '90% fertig in ' + str(perc90) + ' Tagen'
    
    # vertical dotted line originating at mean value
    plt.axvline(perc90+.1, linestyle='--', linewidth=1, color='red', label=text90)

    text80 = '80% fertig in ' + str(perc80) + ' Tagen'
    
    # vertical dotted line originating at mean value
    plt.axvline(perc80+.1, linestyle='-.', linewidth=1, color='blue', label=text80)
    
    #gute Idee, aber das ganze Array muss ersetzt werden
    #if (ax.get_yticks < 1): ax.set_yticks = 1
    
    # Beschriftung
    plt.suptitle(art)
    #raus = str(von) +' bis ' + str(bis) + ': ' + str(len(ldf) + ' Zettel'
    title = von +' bis ' + bis + ': ' + str(len(ldf)) + ' Zettel'
    plt.title(title)
    plt.xlabel('Dauer in Tagen')
    plt.ylabel('Anzahl')
    plt.legend(loc=0)
    
    #plt.show()
    filename='out/'+ art +'.png'
    plt.savefig(filename, dpi=300)
    plt.close()
#enddef plotte()

von=(min(df.input))
von=von.strftime('%d.%m.%Y')

bis=(max(df.out))
bis=bis.strftime('%d.%m.%Y')

for gew_art in df.art.drop_duplicates():
    gewaehlt = df.art == gew_art
#   gewaehlt = df.input > '2018-12-31'
    plot_leadtime(df[gewaehlt], gew_art, von, bis)
#endfor gew_art in ....

plot_leadtime(df, 'Alle', von, bis)

In [7]:
#alle Arbeitstage von ... bis
bdate = pd.bdate_range(von, bis, holidays=feiertage, freq='C')
Tage = bdate.strftime('%d.%m.%y')

# erste Spalte des neuen Dataframes perc
perc = pd.DataFrame(bdate, columns=['days'])
perc['Tage'] = Tage

In [26]:
i = 0
sum = 0
col = np.zeros(len(perc), int) 
for day in perc.days:
    sum += len(df[df.out == day])
    col[i] = sum
    i += 1
#endfor day in perc.days
perc['fertig'] = col

i = 0
sum = 0
col = np.zeros(len(perc), int) 
for day in perc.days:
    sum += len(df[df.input == day])
    sum -= len(df[df.out == day])
    col[i] = sum
    if (col[i] < 0): col[i] = 0
    i += 1
#endfor day in perc.days
perc['Umsetzung'] = col
perc

perc.head()

,days,Tage,fertig,Umsetzung
0,2018-05-29,29.05.18,0,1
1,2018-05-30,30.05.18,0,1
2,2018-05-31,31.05.18,0,1
3,2018-06-01,01.06.18,0,6
4,2018-06-04,04.06.18,0,6


In [46]:
# gca stands for 'get current axis'
ax = plt.gca()
art = 'CFD'

# Beschriftung
plt.suptitle(art)
#raus = str(von) +' bis ' + str(bis) + ': ' + str(len(ldf) + ' Zettel'
#title = von +' bis ' + bis + ': ' + str(len(ldf)) + ' Zettel'
#plt.title(title)
#plt.xlabel('Dauer in Tagen')
plt.ylabel('Anzahl')
#plt.legend(loc=0)

#ax.xaxis_date()


# nur jeden 10. Wert beschriften 
start, end = ax.get_xlim()
print (start, end, np.arange(start, end, 100))
ax.xaxis.set_ticks(np.arange(start, end, 10))

#import matplotlib.dates as mdates
#ax.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')


filename='out/'+ art +'.png'

perc.plot(kind='bar', stacked=True,  linewidth=.1 , x='Tage', y=[2,3], ax=ax)


#fig.autofmt_xdate()

# nur jeden 10. Wert beschriften (zweimal notwendig, aber warum?)
start, end = ax.get_xlim()
print (start, end, np.arange(start, end, 10)*10)
ax.xaxis.set_ticks(np.arange(start, end, 10))

# use a more precise date string for the x axis locations in the
# toolbar
#ax.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')

#ax.autoscale_view()
plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right', size = 5)

plt.savefig(filename, dpi=300)
plt.close()

(0.0, 1.0, array([0.]))
(-0.5, 171.5, array([  -5.,   95.,  195.,  295.,  395.,  495.,  595.,  695.,  795.,
        895.,  995., 1095., 1195., 1295., 1395., 1495., 1595., 1695.]))


In [ ]:
Arbeitstage = pd.bdate_range(von, bis, holidays=feiertage, freq='C')
#Tage[] = Arbeitstage.strftime('%d.%m.%Y')
Arbeitstage